<a href="https://colab.research.google.com/github/kundajelab/label_shift_experiments/blob/master/cifar100/CIFAR100_do_label_shift_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [116]:
!wget https://zenodo.org/record/3458609/files/am_cifar100_test_labels.txt.gz?download=1 -O am_cifar100_test_labels.txt.gz
!wget https://zenodo.org/record/3458609/files/am_cifar100_valid_labels.txt.gz?download=1 -O am_cifar100_valid_labels.txt.gz
!wget https://zenodo.org/record/3458609/files/am_cifar100_validpreacts_seed0.txt.gz?download=1 -O am_cifar100_validpreacts_seed0.txt.gz
!wget https://zenodo.org/record/3458609/files/am_cifar100_testpreacts_seed0.txt.gz?download=1 -O am_cifar100_testpreacts_seed0.txt.gz
!wget https://zenodo.org/record/3458609/files/am_cifar100_validpreacts_seed0.txt.gz?download=1 -O am_cifar100_validpreacts_seed10.txt.gz
!wget https://zenodo.org/record/3458609/files/am_cifar100_testpreacts_seed0.txt.gz?download=1 -O am_cifar100_testpreacts_seed10.txt.gz

--2019-09-24 00:56:43--  https://zenodo.org/record/3458609/files/am_cifar100_test_labels.txt.gz?download=1
Resolving zenodo.org (zenodo.org)... 188.184.65.20
Connecting to zenodo.org (zenodo.org)|188.184.65.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46062 (45K) [application/octet-stream]
Saving to: ‘am_cifar100_test_labels.txt.gz’

am_cifar100_test_la 100%[===================>]  44.98K   147KB/s    in 0.3s    

2019-09-24 00:56:45 (147 KB/s) - ‘am_cifar100_test_labels.txt.gz’ saved [46062/46062]

--2019-09-24 00:56:45--  https://zenodo.org/record/3458609/files/am_cifar100_valid_labels.txt.gz?download=1
Resolving zenodo.org (zenodo.org)... 188.184.65.20
Connecting to zenodo.org (zenodo.org)|188.184.65.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46020 (45K) [application/octet-stream]
Saving to: ‘am_cifar100_valid_labels.txt.gz’

am_cifar100_valid_l 100%[===================>]  44.94K   148KB/s    in 0.3s    

2019-09-24 00

In [126]:
![[ -e abstention ]] || git clone https://github.com/blindauth/abstention
%cd /content/abstention
!git pull
!pip uninstall abstention
!pip install .
%cd ..

/content/abstention
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 25 (delta 18), reused 19 (delta 12), pack-reused 0
Unpacking objects: 100% (25/25), done.
From https://github.com/blindauth/abstention
   e37b8ed..d19d12a  master     -> origin/master
Updating e37b8ed..d19d12a
Fast-forward
 abstention/figure_making_utils.py | 20 ++++++++++----------
 1 file changed, 10 insertions(+), 10 deletions(-)
Uninstalling abstention-0.1.3.0:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/abstention-0.1.3.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/abstention/*
Proceed (y/n)? y
  Successfully uninstalled abstention-0.1.3.0
Processing /content/abstention
  Created wheel for abstention: filename=abstention-0.1.3.0-cp36-none-any.whl size=46751 sha256=e952a13c9e5bfa1ace1521135842c8240c2e79d27bcbffea14622c60c35fec1a
  Stored in directory: /tmp/pip-ephem-wheel-cache-r45p0l0r/whee

/content


In [127]:
![[ -e label_shift_experiments ]] || git clone https://github.com/blindauth/labelshiftexperiments
%cd /content/labelshiftexperiments
!git pull
!pip uninstall labelshiftexperiments
!pip install .
%cd ..

fatal: destination path 'labelshiftexperiments' already exists and is not an empty directory.
/content/labelshiftexperiments
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 8 (delta 6), reused 5 (delta 3), pack-reused 0
Unpacking objects: 100% (8/8), done.
From https://github.com/blindauth/labelshiftexperiments
   f642d16..781f678  master     -> origin/master
Updating f642d16..781f678
Fast-forward
 labelshiftexperiments/maketable.py | 32 ++++++++++++++++----------------
 1 file changed, 16 insertions(+), 16 deletions(-)
Uninstalling labelshiftexperiments-0.1.0.0:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/labelshiftexperiments-0.1.0.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/labelshiftexperiments/*
Proceed (y/n)? y
  Successfully uninstalled labelshiftexperiments-0.1.0.0
Processing /content/labelshiftexperiments
  Created wheel for labelshiftexperiments: filen

/content


In [0]:
!rm *.txt
!gunzip *.gz

In [2]:
from importlib import reload
import abstention
reload(abstention)
reload(abstention.calibration)
reload(abstention.label_shift)
reload(abstention.figure_making_utils)
from abstention.calibration import (
    TempScaling, VectorScaling, NoBiasVectorScaling, softmax)
from abstention.label_shift import (EMImbalanceAdapter,
     BBSEImbalanceAdapter, ShiftWeightFromImbalanceAdapter)
import glob
import gzip
import numpy as np
from collections import defaultdict, OrderedDict

import labelshiftexperiments
reload(labelshiftexperiments)
reload(labelshiftexperiments.cifarandmnist)
from labelshiftexperiments import cifarandmnist
        
test_labels = cifarandmnist.read_preds(open("am_cifar100_test_labels.txt"))
valid_labels = cifarandmnist.read_preds(open("am_cifar100_valid_labels.txt"))

imbalanceadaptername_to_imbalanceadapter = {
    'em': EMImbalanceAdapter(),
    'bbse-hard': BBSEImbalanceAdapter(soft=False),
    'bbse-soft': BBSEImbalanceAdapter(soft=True)}

calibname_to_calibfactory = OrderedDict([
    ('None', abstention.calibration.Softmax()),
    ('TS', TempScaling(verbose=False)),
    ('NBVS', NoBiasVectorScaling(verbose=False)),
    ('BCTS', TempScaling(verbose=False,
                         bias_positions='all')),
    ('VS', VectorScaling(verbose=False)),
    #('CVBCTS', CrossValidatedBCTS(verbose=True,
    #                              num_crossvalidation_splits=5,
    #                              betas_to_try=[0.0, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10]
    #                             ))
])

adaptncalib_pairs = [
    ('bbse-hard', 'None'),
    ('bbse-soft', 'None'),
    ('bbse-soft', 'TS'),
    ('bbse-soft', 'NBVS'),
    ('bbse-soft', 'BCTS'),
    ('bbse-soft', 'VS'),  
    #('bbse-soft', 'CVBCTS'),
    ('em', 'None'),
    ('em', 'TS'),
    ('em', 'NBVS'),
    ('em', 'BCTS'),
    ('em', 'VS'),
    #('em', 'CVBCTS'),
]

num_trials = 10
seeds = [0, 10]
dirichlet_alphas_and_samplesize = [(1.0,5000), (1.0,7500), (1.0,10000),
                                   (0.1,5000), (0.1,7500), (0.1,10000)]
tweakone_alphas_and_samplesize = []

print("Dirichlet shift")

(dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals,      
 dirichlet_alpha_to_samplesize_to_baselineacc,                        
 metric_to_samplesize_to_calibname_to_unshiftedvals) =\
  cifarandmnist.run_experiments(
    num_trials=num_trials,
    seeds=seeds,
    alphas_and_samplesize = dirichlet_alphas_and_samplesize,
    shifttype='dirichlet',
    calibname_to_calibfactory=calibname_to_calibfactory,
    imbalanceadaptername_to_imbalanceadapter=
      imbalanceadaptername_to_imbalanceadapter,
    adaptncalib_pairs=adaptncalib_pairs,
    validglobprefix="am_cifar100_validpreacts_seed",
    testglobprefix="am_cifar100_testpreacts_seed",
    valid_labels=valid_labels,
    test_labels=test_labels)

print("Tweak one shift")

(tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals,      
 tweakone_alpha_to_samplesize_to_baselineacc,                        
 _) = cifarandmnist.run_experiments(
    num_trials=num_trials,
    seeds=seeds,
    alphas_and_samplesize = tweakone_alphas_and_samplesize,
    shifttype='tweakone',
    calibname_to_calibfactory=calibname_to_calibfactory,
    imbalanceadaptername_to_imbalanceadapter=
      imbalanceadaptername_to_imbalanceadapter,
    adaptncalib_pairs=adaptncalib_pairs,
    validglobprefix="am_cifar100_validpreacts_seed",
    testglobprefix="am_cifar100_testpreacts_seed",
    valid_labels=valid_labels,
    test_labels=test_labels)



Dirichlet shift
Seed 0
Seed 10
Calibration stats
Metric ece
None 19.904923460202365
TS 3.180844939372239
NBVS 2.5819155783968832
BCTS 3.160146524628403
VS 2.7704098549009437
Metric nll
None 1.713983503964371
TS 1.2788077827896713
NBVS 1.235503186274253
BCTS 1.2298771907225692
VS 1.2318918689906642
Metric jsdiv
None 0.08253496275261306
TS 0.07142113865149374
NBVS 0.034655311296450816
BCTS 0.035514140240964845
VS 0.035232797484643065
On alpha 1.0 sample size 5000
Metric delta_acc
bbse-hard:None 0.14451 +/- 0.006180026826717454
bbse-soft:None 0.14816 +/- 0.005748640113791671
bbse-soft:TS 0.20636000000000002 +/- 0.006901282870826111
bbse-soft:NBVS 0.20314000000000002 +/- 0.006601755747327257
bbse-soft:BCTS 0.2061 +/- 0.006909680470942656
bbse-soft:VS 0.20396999999999998 +/- 0.006764586967591242
em:None 0.12487999999999999 +/- 0.004217043114740345
em:TS 0.21539000000000003 +/- 0.007039953274366469
em:NBVS 0.21062999999999996 +/- 0.0062135213340279985
em:BCTS 0.21501 +/- 0.006953476601104319

In [5]:
import json
import os
file_out = "label_shift_adaptation_results.json"
dict_to_write = {
    "dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals":
     dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    "dirichlet_alpha_to_samplesize_to_baselineacc":
     dirichlet_alpha_to_samplesize_to_baselineacc,
    "metric_to_samplesize_to_calibname_to_unshiftedvals":
     metric_to_samplesize_to_calibname_to_unshiftedvals,
    "tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals":
     tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    "tweakone_alpha_to_samplesize_to_baselineacc":
     tweakone_alpha_to_samplesize_to_baselineacc
}
open(file_out, 'w').write(
    json.dumps(dict_to_write,
               sort_keys=True, indent=4, separators=(',', ': ')))
os.system("gzip -f "+file_out)

0

In [0]:
import gzip
import json
loaded_dicts = json.loads(gzip.open("label_shift_adaptation_results.json.gz").read())
dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals =\
    loaded_dicts['dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals']
dirichlet_alpha_to_samplesize_to_baselineacc =\
    loaded_dicts['dirichlet_alpha_to_samplesize_to_baselineacc']
tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals =\
    loaded_dicts['tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals']
tweakone_alpha_to_samplesize_to_baselineacc =\
    loaded_dicts['tweakone_alpha_to_samplesize_to_baselineacc']
metric_to_samplesize_to_calibname_to_unshiftedvals =\
    loaded_dicts['metric_to_samplesize_to_calibname_to_unshiftedvals']

In [7]:
from importlib import reload
import labelshiftexperiments
reload(labelshiftexperiments)
import labelshiftexperiments.maketable
reload (labelshiftexperiments.maketable)
from labelshiftexperiments.maketable import render_calibration_table

metricname_to_nicename = {'nll': 'nll', 'jsdiv': 'jsdiv', 'ece': 'ECE'}
calibname_to_nicename = {'None': "None", "TS": "TS",
                         "VS":"VS", "NBVS": "NBVS", "BCTS": "BCTS"}
  
from scipy.stats import norm
N = len(seeds)*num_trials
#Using the normal approximation at N=100;
# variance from https://en.wikipedia.org/wiki/Wilcoxon_signed-rank_test
#Note that T = ((N+1)*N/2 - W)/2            
ustat_threshold = ((N*(N+1))/2 - norm.ppf(0.99)*np.sqrt(N*(N+1)*(2*N+1)/6.0))/2.0
  
print(render_calibration_table(
        metric_to_samplesize_to_calibname_to_unshiftedvals=
          metric_to_samplesize_to_calibname_to_unshiftedvals,
        #threshold of 8 comes from table https://www.oreilly.com/library/view/nonparametric-statistics-a/9781118840429/bapp02.xhtml
        #for one-tailed alpha=0.025 and n=10            
        ustat_threshold=ustat_threshold,
        metrics_in_table=['nll', 'jsdiv', 'ece'],
        samplesizes_in_table=['5000', '7500', '10000'],
        calibnames_in_table=['None', 'TS', 'NBVS', 'BCTS', 'VS'],
        metricname_to_nicename=metricname_to_nicename,
        calibname_to_nicename=calibname_to_nicename,
        caption="CIFAR100 Calibration metric differences", label="cifar10calibrationcomparison",
        applyunderline=False))


\begin{table*}
\adjustbox{max width=\textwidth}{
  \centering
  \begin{tabular}{ c | c c c | c c c | c c c }
    \multirow{2}{*}{\begin{tabular}{c}\textbf{Calibration} \\ \textbf{Method} \end{tabular}} & \multicolumn{3}{| c}{nll} & \multicolumn{3}{| c}{jsdiv} & \multicolumn{3}{| c}{ECE}\\
    \cline{2-10}
    & $n$=5000 & $n$=7500 & $n$=10000 & $n$=5000 & $n$=7500 & $n$=10000 & $n$=5000 & $n$=7500 & $n$=10000\\
    \hline
    None & 1.714; 4.0 & 1.714; 4.0 & 1.714; 4.0 & 0.083; 4.0 & 0.083; 4.0 & 0.083; 4.0 & 19.905; 4.0 & 19.905; 4.0 & 19.905; 4.0\\
    TS & 1.279; 3.0 & 1.279; 3.0 & 1.279; 3.0 & 0.071; 3.0 & 0.071; 3.0 & 0.071; 3.0 & 3.181; 2.5 & 3.17; 2.75 & 3.164; 3.0\\
    NBVS & 1.236; 1.9 & 1.231; 2.0 & 1.229; 2.0 & \textbf{0.035; 0.6} & \textbf{0.03; 0.85} & 0.027; 2.0 & \textbf{2.582; 0.05} & \textbf{2.56; 0.05} & \textbf{2.644; 0.0}\\
    BCTS & \textbf{1.23; 0.2} & 1.226; 1.0 & 1.224; 1.0 & 0.036; 1.65 & 0.03; 1.65 & 0.026; 1.0 & 3.16; 2.5 & 3.119; 2.2 & 3.062; 2.0\\
    VS

In [9]:
from labelshiftexperiments.maketable import render_adaptation_table
    
methodgroups = OrderedDict([
  ('em', ['em:None', 'em:TS', 'em:NBVS', 'em:BCTS', 'em:VS']),
  ('bbse', ['bbse-hard:None', 'bbse-soft:None',
            'bbse-soft:TS', 'bbse-soft:NBVS',
            'bbse-soft:BCTS', 'bbse-soft:VS'])])
samplesizes_in_table = ['5000', '7500', '10000']
adaptname_to_nicename = {'em': 'EM',
                         'bbse-soft': 'BBSE-soft',
                         'bbse-hard': 'BBSE-hard'}
calibname_to_nicename = {'None': 'None',
                         'TS': 'TS',
                         'NBVS': 'NBVS',
                         'BCTS': 'BCTS',
                         'VS': 'VS'}

dirichlet_alphas_in_table = ['0.1', '1.0']

print(render_adaptation_table(
    alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    ustat_threshold=ustat_threshold,
    valmultiplier=1.0,
    adaptname_to_nicename=adaptname_to_nicename,
    calibname_to_nicename=calibname_to_nicename,
    methodgroups=methodgroups,
    metric='jsdiv',
    largerisbetter=False,
    alphas_in_table=dirichlet_alphas_in_table,
    samplesizes_in_table=samplesizes_in_table,
    caption="CIFAR10 Metric: JS Divergence, dirichlet shift",
    label="cifar10jsdivdirichletshift",
    applyunderline=False))

print(render_adaptation_table(
    alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    ustat_threshold=ustat_threshold,
    valmultiplier=100,
    adaptname_to_nicename=adaptname_to_nicename,
    calibname_to_nicename=calibname_to_nicename,
    methodgroups=methodgroups,
    metric='delta_acc',
    largerisbetter=True,
    alphas_in_table=dirichlet_alphas_in_table,
    samplesizes_in_table=samplesizes_in_table,
    caption="CIFAR10 Metric: $\\Delta$\\%Accuracy, dirichlet shift",
    label="cifar10deltaaccdirichletshift",
    applyunderline=False))

"""tweakone_alphas_in_table = []

print(render_adaptation_table(
    alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    ustat_threshold=ustat_threshold,
    valmultiplier=1.0,
    adaptname_to_nicename=adaptname_to_nicename,
    calibname_to_nicename=calibname_to_nicename,
    methodgroups=methodgroups,
    metric='jsdiv',
    largerisbetter=False,
    alphas_in_table=tweakone_alphas_in_table,
    samplesizes_in_table=samplesizes_in_table,
    caption="CIFAR10 Metric: JS Divergence, tweakone shift",
    label="cifar10jsdivtweakoneshift",
    applyunderline=False))

print(render_adaptation_table(
    alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    ustat_threshold=ustat_threshold,
    valmultiplier=100,
    adaptname_to_nicename=adaptname_to_nicename,
    calibname_to_nicename=calibname_to_nicename,
    methodgroups=methodgroups,
    metric='delta_acc',
    largerisbetter=True,
    alphas_in_table=tweakone_alphas_in_table,
    samplesizes_in_table=samplesizes_in_table,
    caption="CIFAR10 Metric: $\\Delta$\\%Accuracy, tweakone shift",
    label="cifar10deltaacctweakoneshift",
    applyunderline=False))"""


\begin{table*}
\adjustbox{max width=\textwidth}{
  \centering
  \begin{tabular}{ c | c | c c c | c c c}
    \multirow{2}{*}{\begin{tabular}{c}\textbf{Shift} \\ \textbf{Estimator} \end{tabular}} & \multirow{2}{*}{\begin{tabular}{c}\textbf{Calibration} \\ \textbf{Method} \end{tabular}} & \multicolumn{3}{| c}{$\alpha=0.1$} & \multicolumn{3}{| c}{$\alpha=1.0$}\\ 
    \cline{3-8}
    & & $n$=5000 & $n$=7500 & $n$=10000 & $n$=5000 & $n$=7500 & $n$=10000\\
    \hline
    \hline
    EM & None & 0.234; 4.0 & 0.234; 4.0 & 0.234; 4.0 & 0.239; 4.0 & 0.237; 4.0 & 0.238; 4.0\\
    EM & TS & \textbf{0.12; 1.4} & 0.118; 1.7 & 0.118; 1.7 & \textbf{0.117; 1.15} & \textbf{0.111; 1.6} & \textbf{0.11; 1.5}\\
    EM & NBVS & \textbf{0.115; 1.75} & 0.112; 1.95 & 0.111; 1.85 & 0.125; 2.25 & 0.114; 2.0 & 0.114; 2.3\\
    EM & BCTS & \textbf{0.112; 1.45} & \textbf{0.109; 1.35} & \textbf{0.11; 1.5} & \textbf{0.119; 1.25} & \textbf{0.11; 1.45} & \textbf{0.11; 1.2}\\
    EM & VS & \textbf{0.11; 1.4} & \textbf{0.1

'tweakone_alphas_in_table = []\n\nprint(render_adaptation_table(\n    alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals,\n    ustat_threshold=ustat_threshold,\n    valmultiplier=1.0,\n    adaptname_to_nicename=adaptname_to_nicename,\n    calibname_to_nicename=calibname_to_nicename,\n    methodgroups=methodgroups,\n    metric=\'jsdiv\',\n    largerisbetter=False,\n    alphas_in_table=tweakone_alphas_in_table,\n    samplesizes_in_table=samplesizes_in_table,\n    caption="CIFAR10 Metric: JS Divergence, tweakone shift",\n    label="cifar10jsdivtweakoneshift",\n    applyunderline=False))\n\nprint(render_adaptation_table(\n    alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals,\n    ustat_threshold=ustat_threshold,\n    valmultiplier=100,\n    adaptname_to_nicename=adaptname_to_nicename,\n    calibname_to_nicename=calibname_to_nicename,\n    methodgroups=methodgrou